# Collect the items - discrete actions

in order to be able to use rllib's DQN is necessary to implement a new version of the environment in which actions are discrete instead of continuous. 

Still the agents' goal is the same as before: collecting all the items in the environment minimizing the number of steps by collaborating together. 

## Preparing the notebook

### import and utils

In [1]:
from utils.vectors import Vector2D
from utils.canvas import CanvasWithBorders
from utils.algo_utils import (save_algo, load_algo)
from utils.simulations import (simulate_episode, simulate_random_episode, dqn_result_format)

from ray.rllib.env.multi_agent_env import MultiAgentEnv
from gymnasium.spaces import Discrete, Box, Dict, Tuple, MultiDiscrete
from gymnasium.spaces.utils import flatten, flatten_space
from IPython.display import clear_output
from ipycanvas import Canvas, hold_canvas
from typing import Set
import random as rnd
import numpy as np
import math

### defining the environment

In [11]:
class EnvironmentConfiguration: 
    def __init__(self, n_agents, n_targets, agent_range, movement_granularity, speed_granularity, spawn_area=100, visible_nbrs=1, visible_targets=1, max_steps=None, cache_size=1):
        # parameters that shouldn't affect the agents' behaviour
        self.n_agents = n_agents
        self.n_targets = n_targets
        self.spawn_area = spawn_area
        self.max_steps = max_steps
        # parameters that affect the agents' behavious
        self.agent_range = agent_range
        # parameters that affect the observation space
        self.visible_nbrs = visible_nbrs
        self.visible_targets = visible_targets
        self.cache_size = cache_size
        # parameters that affect the action space
        self.movement_granularity = movement_granularity
        self.speed_granularity = speed_granularity

    def __deepcopy__(self, memo):
        return EnvironmentConfiguration(
            self.n_agents, 
            self.n_targets, 
            self.agent_range, 
            self.movement_granularity, 
            self.speed_granularity, 
            self.spawn_area, 
            self.visible_nbrs, 
            self.visible_targets, 
            self.max_steps, 
            self.cache_size
        )

In [3]:
class CollectTheItems(MultiAgentEnv):
    canvas = None
    CANVAS_WIDTH, CANVAS_HEIGHT = 300.0, 300.0

    def __init__(self, config: EnvironmentConfiguration):
        assert config.n_agents > config.visible_nbrs
        assert config.movement_granularity % 2 == 1

        self.n_agents = config.n_agents
        self.n_targets = config.n_targets
        self.spawn_area = config.spawn_area
        self.max_steps = config.max_steps
        self.agent_range = config.agent_range
        self.visible_nbrs = config.visible_nbrs
        self.visible_targets = config.visible_targets
        self.cache_size = config.cache_size
        self.movement_granularity = config.movement_granularity
        self.speed_granularity = config.speed_granularity

        self.agents_ids = ['agent-' + str(i) for i in range(self.n_agents)]
        self.observation_space = self.observation_space('agent-0')
        self.action_space = self.action_space('agent-0')

    def unflatten_observation_space(self, agent):
        direction = Box(low=-1, high=1, shape=(2,1), dtype=np.float32)
        distance = Box(low=-np.inf, high=np.inf, shape=(1,1), dtype=np.float32)

        nbrs = Dict({f"nbr-{i}": Dict({'direction': direction, 'distance': distance}) for i in range(self.visible_nbrs)})
        targets = Dict({f"target-{i}": Dict({'direction': direction, 'distance': distance}) for i in range(self.visible_targets)})

        time_t_obs = Dict({"nbrs": nbrs, "targets": targets})

        return Dict({f"t[-{t}]": time_t_obs for t in range(0, self.cache_size)})

    def observation_space(self, agent):
        return flatten_space(self.unflatten_observation_space(agent))

    def __continuous_action(self, discrete_action):
        action_tuple = (discrete_action // (self.movement_granularity*self.speed_granularity), 
                        (discrete_action % (self.movement_granularity*self.speed_granularity)) // (self.speed_granularity), 
                        discrete_action % self.speed_granularity)

        return [(2*(action_tuple[0] / (self.movement_granularity-1))-1),
                (2*(action_tuple[1] / (self.movement_granularity-1))-1),
                (action_tuple[2]) / float(self.speed_granularity-1)]

    def action_space(self, agent):
        """
        direction_x = Discrete(self.movement_granularity)#Box(low=-1.0, high=1.0, shape=(2,1), dtype=np.float32)
        direction_y = Discrete(self.movement_granularity)
        speed = Discrete(self.speed_granularity)#Box(0.0, 1.0, dtype=np.float32)
        return Tuple([direction_x, direction_y, speed])
        """
        return Discrete(self.movement_granularity * self.movement_granularity * self.speed_granularity)
    
    def __get_time_t_observation(self, agent):
        nbrs_distance_vectors = [Vector2D.distance_vector(self.agents_pos[agent], self.agents_pos[nbr])  
                            for nbr in self.__get_n_closest_neighbours(agent, self.visible_nbrs)]

        targets_distance_vectors = [Vector2D.distance_vector(self.agents_pos[agent], self.targets_pos[target])  
                            for target in self.__get_n_closest_targets(agent, self.visible_targets)]

        nbrs = {
            f"nbr-{i}": {
                "direction": Vector2D.unit_vector(nbrs_distance_vectors[i]).to_np_array(),
                "distance": np.log(1 + Vector2D.norm(nbrs_distance_vectors[i])) #1 - np.exp(-alpha * x)
            }
            for i in range(len(nbrs_distance_vectors))
        }
    
        targets = {
            f"target-{i}": {
                "direction": Vector2D.unit_vector(targets_distance_vectors[i]).to_np_array(),
                "distance": np.log(1 + Vector2D.norm(targets_distance_vectors[i])) #1 - np.exp(-alpha * x)
            }
            for i in range(len(targets_distance_vectors))
        }
        
        for i in range(len(targets_distance_vectors), self.visible_targets):
            targets[f"target-{i}"] = {
                "direction": np.array([0,0], dtype=np.int32),
                "distance": -1 #1 - np.exp(-alpha * x)
            }

        obs = {
            "nbrs": nbrs,
            "targets": targets
        }

        return obs

    def __get_observation(self, agent):
        if len(self.observation_cache[agent]) == 0:
            self.observation_cache[agent] = [self.__get_time_t_observation(agent)]*self.cache_size
        else:
            self.observation_cache[agent] = [self.__get_time_t_observation(agent)] + self.observation_cache[agent]
            self.observation_cache[agent].pop()

        obs = {
            f"t[-{t}]": self.observation_cache[agent][t]
            for t in range(0, self.cache_size)
        }

        return flatten(self.unflatten_observation_space(agent), obs)

    def rgb_to_hex(self, r, g, b):
        return f'#{r:02x}{g:02x}{b:02x}'

    def __get_local_reward(self, agent, action):
        # reward_1: small bonus if the agent collects an item
        reward_1 = +5 if agent in self.collectors else 0

        # reward_2: malus if the agent collides with another agent 
        reward_2= sum([-2 if Vector2D.distance(self.agents_pos[agent], self.agents_pos[nbr]) < self.agent_range*2 else 0 for nbr in self.__get_other_agents(agent)])

        # reward_3: -1 at each step
        reward_3 = -1

        # reward_4: positive reward if the agent moves toward the closest targets, negative otherwise
        distance_diff = ([Vector2D.distance(self.agent_old_pos[agent], self.targets_pos[target]) -
                    Vector2D.distance(self.agents_pos[agent], self.targets_pos[target])
            for target in self.closest_targets[agent]])
        
        reward_4 = max(distance_diff) if len(distance_diff) > 0 else 0

        self.info[agent] = {"info": {f"r2: {reward_2}, r3: {reward_3}, r4: {reward_4}"}}
        return  reward_2 + reward_3 + reward_4*3

    def __get_global_reward(self):
        return self.global_reward * 100
    
    def __get_other_agents(self, agent):
        return [other for other in self.agents_ids if other != agent]

    def __get_n_closest_neighbours(self, agent, n=1):
        distances = {other: Vector2D.distance(self.agents_pos[agent], self.agents_pos[other]) for other in self.__get_other_agents(agent)}
        return [neighbour[0] for neighbour in sorted(list(distances.items()), key=lambda d: d[1])[:n]]
        # return {neighbour[0]: neighbour[1] for neighbour in sorted(list(dst.items()), key=lambda d: d[0])[:n]}

    def __get_n_closest_targets(self, agent, n=1):
        n = min(n, len(self.targets_pos.keys()))
        distances = {target: Vector2D.distance(self.agents_pos[agent], pos) for target, pos in self.targets_pos.items()}
        self.closest_targets[agent] = [target[0] for target in sorted(list(distances.items()), key=lambda d: d[1])[:n]]
        return self.closest_targets[agent]

    def __update_agent_position(self, agent, action):
        unit_movement = Vector2D(action[0], action[1])
        self.agent_old_pos[agent] = self.agents_pos[agent]
        self.agents_pos[agent] = Vector2D.sum(self.agents_pos[agent], Vector2D.mul(unit_movement, action[2]))

    def __collect_items(self):
        self.collectors = []
        uncollected_targets = {}
        for target, target_pos in self.targets_pos.items():
            collected = False
            for agent in self.agents_pos.values():
                if Vector2D.distance(target_pos, agent) < self.agent_range:
                    collected = True
                    self.collectors.append(agent)
            if not collected:
                uncollected_targets[target] = target_pos
        self.targets_pos = uncollected_targets

    def __collect_items_and_compute_global_reward(self):
        old_uncollected_items = len(self.targets_pos.keys())
        self.__collect_items()
        updated_uncollected_items = len(self.targets_pos.keys())
        self.global_reward = old_uncollected_items - updated_uncollected_items

    def reset(self, seed=None, options=None):
        self.steps = 0
        self.agents_pos = {agent: Vector2D.get_random_point(max_x=self.spawn_area, max_y=self.spawn_area) for agent in self.agents_ids}
        self.agent_old_pos = dict(self.agents_pos)
        self.targets_pos = {f"target-{i}": Vector2D.get_random_point(max_x=self.spawn_area, max_y=self.spawn_area) for i in range(self.n_targets)}
        self.collectors = []
        self.closest_targets = {}
        self.info = {}
        self.observation_cache = {agent: [] for agent in self.agents_ids}
        return {agent: self.__get_observation(agent) for agent in self.agents_ids}, {}
     
    def step(self, actions):
        self.steps += 1
        observations, rewards, terminated, truncated, infos = {}, {}, {}, {}, {}

        for agent, action in actions.items():
            self.__update_agent_position(agent, self.__continuous_action(action))

        self.__collect_items_and_compute_global_reward()

        for agent, action in actions.items():
            observations[agent] = self.__get_observation(agent)
            rewards[agent] = self.__get_local_reward(agent, self.__continuous_action(action)) + self.__get_global_reward()
            terminated[agent] = False
            truncated[agent] = False
            infos[agent] = self.info[agent]

        truncated['__all__'] = False
        if len(self.targets_pos.keys()) == 0:
            terminated['__all__'] = True
        elif self.max_steps != None and self.steps == self.max_steps:
            terminated['__all__'] = True
        else:
            terminated['__all__'] = False

        return observations, rewards, terminated, truncated, infos
     
    def rgb_to_hex(self, r, g, b):
        return f'#{r:02x}{g:02x}{b:02x}'

    def render(self):
        pass

    def get_agent_ids(self):
       return self.agents


class RenderableCollectTheItems(CollectTheItems):
    def __init__(self, config: EnvironmentConfiguration):
        super().__init__(config)
        self.agent_colors = {agent: self.rgb_to_hex(rnd.randint(0, 255), rnd.randint(0, 255), rnd.randint(0, 255)) for agent in self.agents_ids}

        self.unit = self.CANVAS_WIDTH/float(self.spawn_area)
        self.render_size_agent = max(self.unit,1)
        self.render_size_agent_range = self.unit*self.agent_range

        self.reset()

    def __position_in_frame(self, position_in_env):
        return [((self.spawn_area-position_in_env[0])/self.spawn_area)*self.CANVAS_WIDTH,
                        ((self.spawn_area-position_in_env[1])/self.spawn_area)*self.CANVAS_HEIGHT,]

    def render(self):
        with hold_canvas():
            if self.canvas == None:
                self.canvas = CanvasWithBorders(width=self.CANVAS_WIDTH, height=self.CANVAS_HEIGHT)
                display(self.canvas)

            self.canvas.clear()

            self.canvas.fill_style = "red"
            for target in self.targets_pos.values():
                self.canvas.draw_circle(pos=self.__position_in_frame(target.to_np_array()), 
                                        radius=1, 
                                        fill_color="red")

            for agent in self.agents_ids:
                self.canvas.draw_circle(pos=self.__position_in_frame(self.agents_pos[agent].to_np_array()), 
                                        radius=self.render_size_agent/2.0, 
                                        fill_color=self.agent_colors[agent],
                                        border_color="black")

                self.canvas.draw_circle(pos=self.__position_in_frame(self.agents_pos[agent].to_np_array()), 
                                        radius=self.render_size_agent_range, 
                                        border_color="red")

### environment demonstration

In [4]:
env_config = EnvironmentConfiguration(
    n_agents = 4,
    n_targets = 2,
    spawn_area = 100,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3,
    cache_size=3,
    movement_granularity=5,
    speed_granularity=5)

env = RenderableCollectTheItems(env_config)

print("action space: ", env.action_space)
print("observation space: ", env.unflatten_observation_space("agent-0"))

simulate_random_episode(env, 100, sleep_between_frames=0.03, print_info=False)

action space:  Discrete(125)
observation space:  Dict('t[-0]': Dict('nbrs': Dict('nbr-0': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'nbr-1': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'nbr-2': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32))), 'targets': Dict('target-0': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'target-1': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'target-2': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)))), 't[-1]': Dict('nbrs': Dict('nbr-0': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'nbr-1': Dict('direction': Box(-1.0, 1.0, (2, 1), float32), 'distance': Box(-inf, inf, (1, 1), float32)), 'nbr-2': Dict('d

CanvasWithBorders(height=300, width=300)

## Training

### preparing the environment

In [8]:
from ray.tune.registry import register_env

env_config = EnvironmentConfiguration(
    n_agents = 4,
    n_targets = 10,
    spawn_area = 100,
    max_steps=300,
    agent_range = 5,
    visible_nbrs = 3,
    visible_targets = 3,
    cache_size=3,
    movement_granularity=5,
    speed_granularity=5)

register_env("collect_the_items?algo=DQN&method=CTDE", lambda _: RenderableCollectTheItems(env_config))

### restart ray

In [6]:
import ray
ray.shutdown()

### DQN

In [9]:
from ray.rllib.algorithms.dqn.dqn import DQNConfig

training_iterations = 30

algo = (DQNConfig()
        .training(
            gamma=0.95,
            lr=0.001,
            train_batch_size=32,
            n_step=1,
            target_network_update_freq=500,
            double_q=True,
            dueling=True)
        .env_runners(num_env_runners=1)
        .resources(num_gpus=0)  
        .environment("collect_the_items?algo=DQN&method=CTDE")
    ).build()

clear_output()

out = ""
for i in range(training_iterations):
    result = algo.train()
    clear_output()
    out += dqn_result_format(result) + "\n"
    print(out)
    simulate_episode(RenderableCollectTheItems(env_config), algo, 500, sleep_between_frames=0.01)

iteration [1] => episode_reward_mean: 292.569663468884, episode_len_mean: 300.0
iteration [2] => episode_reward_mean: 514.4395438856817, episode_len_mean: 300.0
iteration [3] => episode_reward_mean: 1224.8661033384365, episode_len_mean: 300.0
iteration [4] => episode_reward_mean: 1601.8209172025317, episode_len_mean: 300.0
iteration [5] => episode_reward_mean: 1648.7731396200656, episode_len_mean: 300.0
iteration [6] => episode_reward_mean: 1969.7434287282315, episode_len_mean: 295.75
iteration [7] => episode_reward_mean: 2103.9601095128724, episode_len_mean: 296.30434782608694
iteration [8] => episode_reward_mean: 2504.4256709625006, episode_len_mean: 282.92857142857144
iteration [9] => episode_reward_mean: 2879.2079598458117, episode_len_mean: 254.28571428571428
iteration [10] => episode_reward_mean: 3132.5300482558982, episode_len_mean: 236.54761904761904
iteration [11] => episode_reward_mean: 3312.7992208001947, episode_len_mean: 218.64
iteration [12] => episode_reward_mean: 3513.2

CanvasWithBorders(height=300, width=300)

## Save and load

In [17]:
save_algo(algo, "collect_the_items?algo=DQN&method=CTDE")

An Algorithm checkpoint has been created inside directory: 'TrainingResult(checkpoint=Checkpoint(filesystem=local, path=/mnt/c/Users/nicol/Desktop/Università/tesi/thesis-MARL_in_Aggregate_Computing/CTDE/algos/collect_the_items?algo=DQN&method=CTDE), metrics={'custom_metrics': {}, 'episode_media': {}, 'info': {'learner': {'default_policy': {'learner_stats': {'allreduce_latency': 0.0, 'grad_gnorm': 4.8280158042907715, 'mean_q': 157.53335571289062, 'min_q': 38.10633087158203, 'max_q': 371.29168701171875, 'cur_lr': 0.001}, 'td_error': array([-1.7874939e+01, -9.1105263e+01, -6.3267975e+00,  8.7053680e+00,
       -8.5085907e+01,  5.4938927e+00, -2.9582428e+01, -3.7129089e+01,
       -7.5896912e+00, -1.1905212e+00, -9.2197113e+01, -3.8870682e+01,
        5.7523041e+00,  2.1991730e+01, -3.7046227e+01,  2.6754852e+01,
        2.2421265e+00,  9.1459351e+00,  6.8840714e+00, -3.3142090e-02,
        1.9923599e+01,  1.0348709e+01,  1.6755657e+01,  4.3000183e+00,
        9.7249298e+00,  2.6508865e+00

In [ ]:
algo = load_algo("collect_the_items?algo=DQN&method=CTDE")

## Simulation

In [16]:
#new_config = env_config.__deepcopy__()
#new_config.spawn_area = 500
#new_config.n_targets = 20
env = RenderableCollectTheItems(env_config)
simulate_episode(env, algo, steps=300, sleep_between_frames=0.03)

CanvasWithBorders(height=300, width=300)

KeyboardInterrupt: 